In [ ]:
!pip install sentence-transformers==3.0.1 scikit-learn==1.5.0 accelerate==0.31.0 gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [204]:
import torch
import pandas as pd
import ast
from sentence_transformers import SentenceTransformer
import pickle
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

In [1]:
print(torch.cuda.is_available())

True


# Process data

In [105]:
MOVIES_PATH = "/content/tmdb_5000_movies.csv"
movies_df = pd.read_csv(MOVIES_PATH)

In [106]:
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [107]:
movies_df.shape

(4803, 20)

In [108]:
processed_df = movies_df.loc[:,["original_title","genres","keywords","overview","popularity"]]

In [109]:
processed_df.head()

,original_title,genres,keywords,overview,popularity
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995


In [110]:
processed_df.isna().sum()

,0
original_title,0
genres,0
keywords,0
overview,3
popularity,0


In [111]:
processed_df.dropna(inplace= True)

In [112]:
processed_df.duplicated().sum().item()

0

In [113]:
processed_df.loc[0,"genres"]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [114]:
def extract_genres(list):
    genre_names = []
    for genre in ast.literal_eval(list):
        genre_names.append(genre["name"])
    return genre_names

In [115]:
def concat (x):
  return  " ".join(x)

In [116]:
extract_genres(processed_df.loc[0,"genres"])

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [117]:
processed_df["genres"] = processed_df["genres"].apply(extract_genres)

In [118]:
processed_df["genres"] = processed_df["genres"].apply(concat)

In [119]:
extract_genres(processed_df.loc[0,"keywords"])

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [120]:
processed_df["keywords"] = processed_df["keywords"].apply(extract_genres)

In [121]:
processed_df["keywords"] = processed_df["keywords"].apply(concat)

In [122]:
processed_df['text'] =processed_df["original_title"] + " " + processed_df["keywords"] + processed_df["genres"] + processed_df["overview"]

In [99]:
processed_df.head()

,original_title,genres,keywords,overview,popularity,text
0,Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",150.437577,Avatar culture clash future space war space co...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",139.082615,Pirates of the Caribbean: At World's End ocean...
2,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,107.376788,Spectre spy based on novel secret agent sequel...
3,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,112.312950,The Dark Knight Rises dc comics crime fighter ...
4,John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",43.926995,John Carter based on novel mars medallion spac...


In [123]:
processed_df  = processed_df.loc[:,["original_title","text","popularity"]]

In [124]:
processed_df.head(1)

,original_title,text,popularity
0,Avatar,Avatar culture clash future space war space co...,150.437577


In [125]:
processed_df.shape

(4800, 3)

# Embeddings using sentence transformers

In [25]:
MODEL_PATH = "sentence-transformers/all-mpnet-base-v2"
Model  =  SentenceTransformer(MODEL_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [102]:
movie_texts = processed_df['text'].tolist()
movie_embeddings = Model.encode(movie_texts, show_progress_bar=True)

Batches:   0%|          | 0/150 [00:00<?, ?it/s]

In [103]:
movie_embeddings.shape

(4800, 768)

#Process Query

In [126]:
with open('movie_embeddings.pkl', 'wb') as f:
    pickle.dump({'embeddings': movie_embeddings, 'titles': processed_df['original_title']}, f)

In [127]:
def load_embeddings(pickle_path):
  with open(pickle_path, 'rb') as f:
    data = pickle.load(f)
    embeddings = data['embeddings']
    titles = data['titles']
  return embeddings, titles

In [137]:
def query_embedding(query):
  return Model.encode(query)


# Semantic Search

In [156]:
scaler = MinMaxScaler()

In [162]:
def semantic_search(query_embedding, top_k=5):
    embeddings, movie_titles = load_embeddings('movie_embeddings.pkl')
    sim_scores = cosine_similarity([query_embedding],embeddings)[0]
    top_indices = np.argsort(sim_scores)[-top_k:][::-1]
    results_df =processed_df.iloc[top_indices][['original_title']]
    results_df['norm_pop'] = scaler.fit_transform(processed_df.iloc[top_indices][['popularity']])
    results_df['sim_scores'] = sim_scores[top_indices]
    results_df['norm_sim'] = scaler.fit_transform(results_df[['sim_scores']])
    return results_df.loc[:,["original_title","norm_pop","norm_sim"]]


In [163]:
query = "space adventure with aliens"
query_em = query_embedding(query)
semantic_search(query_em, top_k=10)

,original_title,norm_pop,norm_sim
3904,Spaced Invaders,0.000000,0.999999
2403,Aliens,1.000000,0.854156
577,AVP: Alien vs. Predator,0.626122,0.806499
4187,Independence Daysaster,0.005642,0.706017
461,Lost in Space,0.240160,0.344255
1198,Escape from Planet Earth,0.250560,0.249755
643,Space Cowboys,0.214690,0.167249
373,Mission to Mars,0.219021,0.023642
539,Titan A.E.,0.194587,0.023159
184,Home,0.936619,0.000000


# Hybrid Search
- Introduce Movie bias by applying weight to the popularity of movie itself

In [168]:
def hybrid_search(sem_results,top_k=5, popularity_weight=0.1):
  sem_results['hybrid_score'] = (1 - popularity_weight) * sem_results['norm_sim'] + (popularity_weight * sem_results['norm_pop'])
  top_indices = np.argsort(sem_results['hybrid_score'])[-top_k:][::-1]
  return sem_results.iloc[top_indices][['original_title']]


In [169]:
query = "space adventure with aliens"
query_em = query_embedding(query)
res = semantic_search(query_em, top_k=10)
hybrid_search(res,top_k=10)

,original_title
3904,Spaced Invaders
2403,Aliens
577,AVP: Alien vs. Predator
4187,Independence Daysaster
461,Lost in Space
1198,Escape from Planet Earth
643,Space Cowboys
184,Home
373,Mission to Mars
539,Titan A.E.


In [199]:
def recommend_movies(query, top_k=5):
    query_em = query_embedding(query)
    sem_res = semantic_search(query_em, top_k*3)
    results = hybrid_search(sem_res, top_k)
    return pd.DataFrame({"Recommended Movies": results["original_title"].tolist()})

In [200]:
recommend_movies("Avatar", top_k=5)

,Recommended Movies
0,Avatar
1,The Last Airbender
2,Creative Control
3,Unfriended
4,The Neon Demon


In [203]:
iface = gr.Interface(
    fn=recommend_movies,
    inputs=[
        gr.Textbox(label="Describe a movie you'd like to see"),
        gr.Slider(1, 10, value=5,step=1, label="Number of recommendations")
    ],
    outputs=gr.Dataframe(
        headers=["Recommended Movies"],
        datatype=["object"]
    ),
    title="🎬 Movie Recommender System",
    description="Find similar movies based semantics and popularity"
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bdbca758936889eeff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
